In [12]:
import numpy as np
import pandas as pd
from datetime import datetime, date
import inflection

# Data Preprossesing

### Loading Data

In [13]:
data_train_raw = pd.read_csv( '../data/raw/train.csv', low_memory=False)
data_test_raw = pd.read_csv( '../data/raw/test.csv', low_memory=False)
data_store_raw = pd.read_csv( '../data/raw/store.csv', low_memory=False)

In [14]:
data_train_raw.shape, data_test_raw.shape, data_store_raw.shape

((1017209, 9), (41088, 8), (1115, 10))

In [15]:
data_train_raw = pd.merge( data_train_raw, data_store_raw, how='left', on='Store')

In [16]:
data = data_train_raw.copy()

### Rename Columns

In [17]:
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map( snakecase, data.columns))
data.columns = cols_new

### Dealing with missing values

In [18]:
data.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

In [19]:
#There are several ways to handle missing data. In the case of competition_distance, a high value equivalent to the 
#third quartile was assigned, since not filling this field suggests a lack of knowledge of nearby competitors. 
#In the case of competition time, the median is a safer guess. As for the time of promo2, the missing data mostly 
#indicates that they are really null.

data[['competition_open_since_month','competition_open_since_year']] = data[['competition_open_since_month', 'competition_open_since_year']].fillna(data[['competition_open_since_month', 'competition_open_since_year']].median())
data['competition_distance'] = data['competition_distance'].fillna(7000)                          
data= data.fillna(0)

In [20]:
data['date'] = pd.to_datetime(data['date'])

## Feature Engineering

In [21]:
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data['day_of_year'] = data['date'].dt.dayofyear

In [22]:
# Calculates the time the competitor has been open in months
data['competition_open_since'] = 12 * (data['date'].dt.year - data['competition_open_since_year']) + (data['date'].dt.month - data['competition_open_since_month'])
data['competition_open_since'] = data['competition_open_since'].apply(lambda x: x if x > 0 else 0)

In [23]:
# calculate promo2 open time in weeks
data['promo_open_since'] = 12 * (data['date'].dt.year - data.promo2_since_year) + (data['date'].dt.isocalendar().week - data.promo2_since_week) / 4.0
data['promo_open_since'] = data['promo_open_since'].apply(lambda x: x if x > 0 else 0)

### Rename Row values

In [24]:
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
data['month_map'] = data['date'].dt.month.map(month_map)

#Analise se está em promoção ou não com base nos meses de promoção
data['is_promo_month'] =data[['promo_interval','month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 \
                                                             else 1 if x['month_map'] in x['promo_interval'].split(',') \
                                                             else 0, axis=1)

In [26]:
data=data.drop(['month_map'], axis=1)

### Dealing with miss inputation value

In [25]:
# Over 1115 stores, store "262" was the only one that was considering that Easter is not a school holiday, I assume this is a mistake
data.loc[(data['school_holiday']==0) & (data['state_holiday']=='b'), 'school_holiday'] = 1

### Fixing Data Types

In [27]:
data = data.astype({"store": 'int32', 
                    "day_of_week": 'int8',
                    "customers": 'int32',
                    "open": 'int8',
                    "promo": 'int8',
                    "state_holiday": 'category',
                    "school_holiday": 'int8',
                    "store_type": 'category',
                    "assortment": 'category', 
                    "competition_distance": 'int32',
                    "competition_open_since_month": 'int32',
                    "competition_open_since_year": 'int32',
                    "promo2": 'int8',
                    "promo2_since_week": 'int32',
                    "promo2_since_year": 'int32',
                    "day": 'int8', 
                    "month": 'int32', 
                    "year": 'int32',
                    "day_of_year": 'int32',
                    "competition_open_since": 'int32',
                    "promo_open_since": 'int32',
                    "is_promo_month" : 'int8'
                   })

# This step reduce memory usage from 162MB to 66.9MB

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 25 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   store                         1017209 non-null  int32         
 1   day_of_week                   1017209 non-null  int8          
 2   date                          1017209 non-null  datetime64[ns]
 3   sales                         1017209 non-null  int64         
 4   customers                     1017209 non-null  int32         
 5   open                          1017209 non-null  int8          
 6   promo                         1017209 non-null  int8          
 7   state_holiday                 1017209 non-null  category      
 8   school_holiday                1017209 non-null  int8          
 9   store_type                    1017209 non-null  category      
 10  assortment                    1017209 non-null  category      
 11